In [2]:
import torch
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import torch.nn.init

In [ ]:
inputs = torch.Tensor(1, 1, 28, 28) #이미지 수, 채널, 행, 열 -> 이미지 한장
inputs.size()

torch.Size([1, 1, 28, 28])

컨볼루션 연산을 위해 값이 어떻게 들어가있는지 보자

In [4]:
inputs

tensor([[[[1.5977e+29, 1.1589e-42, 0.0000e+00, 0.0000e+00, 0.0000e+00,
           0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
           0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
           0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
           0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
           0.0000e+00, 0.0000e+00, 0.0000e+00],
          [0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
           0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
           0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
           0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
           0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
           0.0000e+00, 0.0000e+00, 0.0000e+00],
          [0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
           0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
           0.0000e+00, 0.0000e+00, 0.0000e+00, 0.000

nn 이라는 라이브러리 안에 들어있어

In [ ]:
conv1=nn.Conv2d(1, 32, 3, padding=1, stride=1)
 #채널 수, 출력채널 수, 필터 수, padding 1칸씩, stride=1 이거 안줘도 기본임
print(conv1)

Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))


위 출력이 32였으니까, 입력이 32, 커널사이즈 동일, 패딩은 1
패딩은 생략하면 기본 제공 아니니 꼭 넣자. stride 는 생략해도 1 들어가.
1칸 이동이 기본 설정

3*3의 커널을 32개 만들어라.

In [6]:
conv2=nn.Conv2d(32, 64, kernel_size=3, padding=1)
print(conv2)

Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))


32개를 받아서 64개를 만들어라.
커널이 64개.

pool

In [ ]:
pool=nn.MaxPool2d(2) # 2*2 maxpooling 하면 겹치는게 없어서?? 다음칸으로 이동 stride=2
print(pool)

MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)


In [8]:
out1=conv1(inputs)
out2=conv2(out1) #첫번재 컨볼루션을 넣겠다.
print(out1.size())
print(out2.size())

torch.Size([1, 32, 28, 28])
torch.Size([1, 64, 28, 28])


torch.Size([1, 32, 28, 28]) 입력 출력 이미지 크기 28*28
torch.Size([1, 64, 28, 28]) 입력 출력 이미지 크기 28*28

풀링을 적용해보자.

In [9]:
out1=conv1(inputs)
out2=pool(out1)
# out2=conv2(out1) #첫번재 컨볼루션을 넣겠다.
print(out1.size())
print(out2.size())

torch.Size([1, 32, 28, 28])
torch.Size([1, 32, 14, 14])


torch.Size([1, 32, 28, 28]) 입력 출력 이미지 28*28
torch.Size([1, 32, 14, 14]) pooling 하니 크기가 14*14 으로 절반 작아짐

In [10]:
out3=conv2(out2)
out4=pool(out3)
print(out3.size())
print(out4.size())

torch.Size([1, 64, 14, 14])
torch.Size([1, 64, 7, 7])


torch.Size([1, 64, 14, 14]) #출력이 64 
torch.Size([1, 64, 7, 7])

컨볼루션 연산 2단을 실행한 거임.

In [11]:
out=out4.view(out4.size(0), -1) # out4 하나로 만든다.
print(out.size())

torch.Size([1, 3136])


3136개의 데이터가 생김 -> 이 값의 결과로 분류를 한다.
분류를 위해 1차원으로 만들었어.
여러층을 내든, 소프트맥스를 하든 특징값을 찾아.

In [12]:
import numpy as np
fc = nn.Linear(64*7*7, 10) #엠니트스라면 출력값은 10가지

outf=fc(out) #out을 fc에 넣는다. 리니어리그레이션 한다.

print(outf.size())
print(outf)

torch.Size([1, 10])
tensor([[ 9.2802e+26, -4.6891e+25, -1.1001e+27,  4.7330e+26, -5.3280e+26,
         -6.8008e+26, -1.3528e+26, -1.5939e+26,  5.5330e+25, -1.6921e+26]],
       grad_fn=<AddmmBackward0>)


임의의 수 10개가 나왔어

In [13]:
index=torch.argmax(outf)
print(index)

tensor(0)


가장 큰 값의 인덱스 0을 가지고 왔어.